In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 33.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a0690b3ab6d3d6116196aad469be789734d560e9f432fccd5b8ac7e5167f6bf3
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Logistic Regression Model").getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.sql import functions as f
import time

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [4]:
df = df.drop('objid', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z| redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  2

In [5]:
print(df.count(), len(df.columns))

3524477 27


In [6]:
### Remove Duplicate rows in Dataframe
df = df.dropDuplicates()
print(df.count(), len(df.columns))

3524477 27


In [7]:
import pyspark.sql.functions as func

df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [8]:
import pandas as pd
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
modelMag_u,21.322890,21.540440,24.099690,19.703640,21.891370
modelMag_g,21.199790,21.344430,21.893860,17.857980,19.901890
modelMag_r,20.978130,20.455390,20.295770,16.785280,18.379160
modelMag_i,20.786010,19.635970,19.441590,16.337970,17.835950
modelMag_z,20.602870,19.201110,19.201700,16.005240,17.496330
fiberMag_u,21.721530,22.420790,24.791400,20.956700,22.894670
fiberMag_g,21.571780,22.167020,22.645580,19.119800,20.870210
fiberMag_r,21.418020,21.318560,21.237360,18.061850,19.368600
fiberMag_i,21.290620,20.557740,20.370210,17.609200,18.812600
fiberMag_z,21.389900,20.050190,20.087840,17.276210,18.468090


In [9]:
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+-

In [10]:
df.columns

['modelMag_u',
 'modelMag_g',
 'modelMag_r',
 'modelMag_i',
 'modelMag_z',
 'fiberMag_u',
 'fiberMag_g',
 'fiberMag_r',
 'fiberMag_i',
 'fiberMag_z',
 'petroR50_r',
 'petroR90_r',
 'petroR50_z',
 'petroR90_z',
 'r',
 'i',
 'z',
 'redshift',
 'zerr',
 'mmug',
 'mmgr',
 'mmri',
 'mmiz',
 'mfug',
 'mfgr',
 'mfri',
 'mfiz']

In [11]:
#df.show(5)

In [12]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+-----+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|objid|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------

In [13]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [14]:
output = assembler.transform(df)

In [15]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[21.54044,21.3444...|       0|
|[24.09969,21.8938...|       0|
+--------------------+--------+
only showing top 3 rows



In [16]:
train, test = final_df.randomSplit([0.7, 0.3])

In [17]:
classifier = LogisticRegression(labelCol = 'redshift', featuresCol = 'features')

In [18]:
start_time = time.time()
model = classifier.fit(train)
print("%s seconds" % (time.time() - start_time))

474.5162992477417 seconds


In [20]:
lr_train_prediction2 = model.transform(train)
lr_train_prediction2.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[12.35487,13.6287...|       0|[68.0472584825650...|[0.99999999999522...|       0.0|
|[13.68416,11.8890...|       0|[40.9622707660363...|[0.99999999994851...|       0.0|
|[13.78523,12.4974...|       0|[39.3027202138704...|[0.99999999961124...|       0.0|
|[13.8472700000000...|       0|[81.0823994103138...|[0.99999999999999...|       0.0|
|[14.02077,12.3308...|       0|[58.8996972862988...|[0.99999999999932...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Random Forest Training Accuracy:', multi_evaluator.evaluate(lr_train_prediction2))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(lr_train_prediction2))

Random Forest Training Accuracy: 0.8912952210518809
Training data (weightedPrecision):  0.8892387868255229


In [22]:
pred_data = model.transform(test)

In [23]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')

In [24]:
AUC = evaluator.evaluate(pred_data)

In [25]:
print(AUC)

0.8921312123812831


In [26]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data))

test data (weightedPrecision):  0.8901184822174969
test data (weightedRecall):  0.8921312123812831


In [27]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol = 'redshift')
predictionAndTarget = model.transform(final_df).select("redshift", "prediction")
acc = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "accuracy"})
print(acc)

0.8915458946107465


# Cascading

In [28]:
pred_data2 = model.transform(final_df)

In [29]:
pred_data2.count()

3524477

In [30]:
pred_data2.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.32289,21.1997...|       2|[2.76298995723503...|[0.11722997883591...|       1.0|
|[21.54044,21.3444...|       0|[6.53456712453144...|[0.92840773273088...|       0.0|
|[24.09969,21.8938...|       0|[6.71072205149065...|[0.99496009560922...|       0.0|
|[19.70364,17.8579...|       0|[8.58763705867892...|[0.99751602303111...|       0.0|
|[21.89137,19.9018...|       0|[7.34969455745316...|[0.99666288497561...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [31]:
from pyspark.sql.functions import monotonically_increasing_id
pred_data2 = pred_data2.withColumn("objid", monotonically_increasing_id())
pred_data2.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.32289,21.1997...|       2|[2.76298995723503...|[0.11722997883591...|       1.0|    0|
|[21.54044,21.3444...|       0|[6.53456712453144...|[0.92840773273088...|       0.0|    1|
|[24.09969,21.8938...|       0|[6.71072205149065...|[0.99496009560922...|       0.0|    2|
|[19.70364,17.8579...|       0|[8.58763705867892...|[0.99751602303111...|       0.0|    3|
|[21.89137,19.9018...|       0|[7.34969455745316...|[0.99666288497561...|       0.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [32]:
print(df.count(), len(df.columns))
print(pred_data2.count(), len(pred_data2.columns))

new_df = df.join(pred_data2, df.objid == pred_data2.objid).select(df["*"],pred_data2["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

3524477 28
3524477 6
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+---------+------------------+---------+------------------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|        petroR90_z|       r|       i|                 z|redshift|       zerr|     mmug|     mmgr|              mmri|     mmiz|              mfug|     mfgr|     mfri|    mfiz|objid|prediction|
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+----

In [33]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [34]:
output = assembler.transform(new_df)

In [35]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[20.66357,20.0293...|       0|
|[21.01588,20.8195...|       1|
+--------------------+--------+
only showing top 3 rows



In [36]:
train, test = final_df.randomSplit([0.7, 0.3])

In [37]:
classifier = LogisticRegression(labelCol = 'redshift',featuresCol = 'features2')

In [38]:
model2 = classifier.fit(train)

In [39]:
lr_train_prediction2 = model2.transform(train)
lr_train_prediction2.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|           features2|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[10.54181,12.6374...|       0|[70.5006220995946...|[0.99999999999999...|       0.0|
|[12.3422,13.32923...|       6|[62.3614594779038...|[0.99999999999948...|       0.0|
|[12.75383,11.8222...|       0|[56.9726629332760...|[0.99999999999990...|       0.0|
|[12.75609,13.7903...|       0|[117.912659180852...|[1.0,9.5170431873...|       0.0|
|[12.83451,14.4982...|       1|[140.792405229362...|[1.0,1.0426821608...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [41]:
print('Random Forest Training Accuracy:', multi_evaluator.evaluate(lr_train_prediction2))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(lr_train_prediction2))

Random Forest Training Accuracy: 0.8906008124514255
Training data (weightedPrecision):  0.8906008124514255


In [42]:
pred_data2 = model2.transform(test)

In [43]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')
AUC = evaluator.evaluate(pred_data2)
print(AUC)

0.8911186554895448


In [44]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data2))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data2))

test data (weightedPrecision):  0.8897387197498902
test data (weightedRecall):  0.8911186554895448
